In [1]:
print("Hello World")

Hello World


In [13]:
pip install pandas nltk pyodbc sqlalchemy SQLAlchemy

In [1]:
import pandas as pd
import nltk
import pyodbc
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Download the VADER lexicon for sentiment analysis if not already present
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [11]:
# define a function to fetch data from the SQL database using SQL query
def fetch_data_from_sql():
    # Define th connection string with parameters for the database connection
    conn_str = (
        "DRIVER={SQL Server};" # specify the driver for sql server
        "Server=HP\\SQLEXPRESS;" # specify your SQL Server Instance
        "Database=PortfolioProject_MarketingAnalytics;" # specify the database name
        "Trusted_Connection=yes" # use windows authentication for the connection
    )
    # Establish the connection to the database
    conn = pyodbc.connect(conn_str)

    # Define the sql query to fetch customer reviews data
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"

    # execute the query and fetch the data into a DataFrame
    df = pd.read_sql(query, conn)

    # close the connection to free up connetion resources
    conn.close()

    # return the fetched data as a DataFrame
    return df

In [15]:
# fetch the customer reviews data from the database
customer_review_df = fetch_data_from_sql()

C:\Users\user\AppData\Local\Temp\ipykernel_4768\2056898511.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [17]:
# initialize the VADER sentiment intensity analuzer for analysing the sentiment of the text data
sia = SentimentIntensityAnalyzer()

# Define a function to calculate the sentiment score using VADER
def calculate_sentiment(review):
    # get the sentiment score for the review test
    sentiment = sia.polarity_scores(review)
    # return the compound score, which is a normalized score betwenn -1(most negative) and 1(ost positive)
    return sentiment['compound']

In [27]:
# define a function to categorize sentiments using both sentiment score and the review score
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'  # high rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive' # netural rating and positive sentiment
        else:
            return 'Mixed Negative' # low rating and positive sentiment
    elif score < -0.05:
        if rating <= 2:
            return 'Negative' # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative' #netural rating and negative sentiment
    else:
        if rating >=4:
            return 'Positive'
        elif rating <=2:
            return 'Negative'
        else:
            return 'Netural'

In [28]:
# define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.05:
        return "0.5 to 1.0" # strongly positive sentment
    elif 0.0 <= score < 0.5:
        return "0.0 to 0.5"  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return "-0.49 to 0.0" # mildly negative sentiment
    else:
        return "-1.0 to -0.5" # strongly negative sentiment

In [29]:
# apply sentiment analysis to calculate sentiment scores to each rows
customer_review_df['SentimentScore'] = customer_review_df['ReviewText'].apply(calculate_sentiment)

# apply sentiment categorizatio to both text and rating
customer_review_df['SentimentCategory'] = customer_review_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

# applying sentiment bucketing to categorze acores into defined ranges
customer_review_df['SentimentBucket'] = customer_review_df['SentimentScore'].apply(sentiment_bucket)


In [31]:
# disply the first few rows of the dataFrame with sentiment scores, categories and buckets
customer_review_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.5
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.5
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.5 to 1.0
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [33]:
# save the DataFrame with sentiment score, categories and buckets to a new CSV file
customer_review_df.to_csv('fact_customer_reviwes_with_sentiment.csv', index=False)